In [13]:
import warnings
warnings.filterwarnings("ignore")
import torch
import sys

import tgt
import os
import numpy as np
from tqdm import tqdm
import torchaudio
import librosa
import glob

sys.path.append("/mntcephfs/lab_data/shoinoue/Models/trained_models/vocos/vocos16k_noncausal_tealab/")
from vocos16k_inference import Vocos

sys.path.append("../../cuhksz-phd/sho_util/pyfiles/")
from pytorch import cuda2numpy, cuda2cpu
from basic import plot_spectrogram
from sound import play_audio

from vocos.feature_extractors import MelSpectrogramFeatures, EncodecFeatures

################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
################################################################################



In [14]:
data_dir = "/mntcephfs/lab_data/shoinoue/"
dataset_dir = data_dir + "Dataset/L2-ARCTIC/"
fs = 24000

melgen = MelSpectrogramFeatures(sample_rate=fs)
speakers = [os.path.basename(a) for a in glob.glob(dataset_dir + "*")]
speakers.sort()

In [27]:
save = False

for spk in speakers:
    print(spk)
    mel_dir = f"{dataset_dir}{spk}/mel/"
    os.makedirs(mel_dir, exist_ok=True)
    filenames = glob.glob(dataset_dir + spk + "/wav/*")
    filenames.sort()
    for path in tqdm(filenames):
        y, sr = torchaudio.load(path)
        if fs!=sr:
            if y.size(0) > 1:  # mix to mono
                y = y.mean(dim=0, keepdim=True)
            y = torchaudio.functional.resample(y, orig_freq=sr, new_freq=fs)
        mel = melgen(y[0])
        savepath = mel_dir + os.path.basename(path)[:-4] + ".npy"
        if save:
            np.save(savepath, mel)

HKK


100%|██████████| 1131/1131 [00:13<00:00, 82.66it/s]


TNI


100%|██████████| 1131/1131 [00:16<00:00, 68.38it/s]


# Normalize

In [43]:
from sklearn.preprocessing import StandardScaler
import numpy as np
import joblib

datasplit = list(np.load("./data_split_ARCTIC.npy", allow_pickle=True))

In [6]:
target_speakers = ["HKK", "TNI"] 

if target_speakers==["HKK", "TNI"]:
    scaler_name = "HKK_TNI"

In [44]:
scaler = StandardScaler()
for spk in target_speakers:
    print(spk)
    mel_dir = f"{dataset_dir}{spk}/mel/"
    files = glob.glob(mel_dir + "*")
    files.sort()
    for path in tqdm(files):
        if os.path.basename(path)[:-4] in datasplit[0]: # sample must be in train
            mel = np.load(path)
            scaler.partial_fit(mel.T)
            
scaler_filename = f"ckpts/scalers/{scaler_name}.save"
joblib.dump(scaler, scaler_filename) 
# a = joblib.load(scaler_filename)

HKK


100%|██████████| 1131/1131 [00:00<00:00, 1592.27it/s]


TNI


100%|██████████| 1131/1131 [00:00<00:00, 1682.23it/s]


['ckpts/scalers/HKK_TNI.save']